In [1]:
import requests
import json
import pandas as pd
import re

In [2]:
FORMATTER_URL = 'https://play.openpolicyagent.org/v1/fmt'
LINT_URL = 'https://play.openpolicyagent.org/v1/lint'
PRIVATEGPT_URL = 'http://localhost:5000/get_answer'

PRIVATEGPT_HEADERS = {"Content-Type": "application/json"}

sentence = 'package abc'

# Data to be sent in the POST request
POST_DATA = {
    "rego_module": sentence
}

In [3]:
# Process OPA format function
def process_fmt(post_data):
    try:
        response = requests.post(FORMATTER_URL, json=post_data)

        if('result' not in response.json()):
            raise requests.RequestException("Result not present in fmt response")
        print(response.json()["result"])

        if response.status_code // 100 == 2:
            print("Formatting successful. Processing lint")
        else:
            print(f'Error making POST request. Status code: {response.status_code}')
            
        return response

    except requests.RequestException as e:
        print('Error:', e)

In [4]:
def process_lint(post_data):
    try:
        response = requests.post(LINT_URL, json=post_data)

        if (response.status_code // 100) != 2:
            print(f'Error making POST request. Status code: {response.status_code}')

        return response
    
    except requests.RequestException as e:
        print('Error:', e)

In [5]:
def process_privategpt(post_data):
    try:
        privategpt_req_body = {"query": post_data}
        privategpt_req_body = json.dumps(privategpt_req_body)
        privategpt_response = requests.post(PRIVATEGPT_URL,json=privategpt_req_body,headers=PRIVATEGPT_HEADERS)
        
        if (privategpt_response.status_code // 100) != 2:
            print(f'Error making POST request. Status code: {privategpt_response.status_code}')

        return privategpt_response
        
    except requests.RequestException as e:
        print('Error:', e)

In [6]:
# Include data fetch code from privategpt server
queries_df = pd.read_csv("queries.csv")
query = queries_df.sample(n=1)
query_placeholder = query["prompt"].iloc[0]

print("Query asked: ",query_placeholder)

response_privategpt = process_privategpt(query_placeholder)

Query asked:  Implement a REGO rule sm1 if rso>puj, vq>ji and t!=x.


In [7]:
print(response_privategpt)

<Response [200]>


In [8]:
print(response_privategpt.json())

{'answer': ' Rule r from above would be implemented as follows using the two variables p (input) and q(output):', 'query': '{"query": "Implement a REGO rule sm1 if rso>puj, vq>ji and t!=x."}', 'source': [{'name': 'source_documents/rego_basic_unlabelled.txt'}, {'name': 'source_documents/rego_basic_unlabelled.txt'}, {'name': 'source_documents/rego_basic_unlabelled.txt'}, {'name': 'source_documents/rego_basic_unlabelled.txt'}]}


In [9]:
privategpt_answer = response_privategpt.json()["answer"]

In [13]:
print(privategpt_answer)

 Rule r from above would be implemented as follows using the two variables p (input) and q(output):
